In [17]:
# Import and load report
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_excel('report.xls')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828 entries, 0 to 827
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Deal        828 non-null    int64  
 1   Open time   828 non-null    object 
 2   Type        828 non-null    object 
 3   Size        828 non-null    float64
 4   Item        814 non-null    object 
 5   Price       828 non-null    float64
 6   S/L         828 non-null    int64  
 7   T/P         828 non-null    float64
 8   Time        828 non-null    object 
 9   Close       828 non-null    float64
 10  Commission  828 non-null    int64  
 11  Swap        828 non-null    float64
 12  Profit      828 non-null    float64
 13  Comment     828 non-null    object 
dtypes: float64(6), int64(3), object(5)
memory usage: 90.7+ KB


In [18]:
# Convert types and describe
CN = {'Open': 'Open time',
      'Close': 'Time',
      'Comment': 'Comment',
      'OrderID': 'Deal',
      'Profit': 'Profit',
      'Side': 'Type',
      'Qty': 'Size',
      
      'OpenPrice': 'Price',
      'TakeProfitPrice': 'T/P',

      'BalanceIn': 'DK_BALANCE_IN',
      'BalanceOut': 'DK_BALANCE_OUT',
      'WorstPrice': 'DK_WORST_PRICE',
      'OpenValue': 'DK_OPEN_VALUE',
      'NetID': 'DK_NET_ID',
      'NetQty': 'DK_NET_QTY',
      'NetValue': 'DK_NET_VALUE',
      'NetLastPrice': 'DK_NET_LAST_PRICE',
      'NetAvgPrice': 'DK_NET_AVG_PRICE',

      'Drawdown': 'DK_DRAWDOWN',
      'DrawdownRatio': 'DK_DRAWDOWN_RATIO',
      }

COMMENT_PATTERN_FOR_ORDERS_ONLY = "[tp]"
OPEN_NEW_ORDER_PRICE_DELTA_PIPS = 180 # Eve average position when price goes more then 180 pips
XAU_PIP_USD = 0.01 # 1 pip = 0.01 for XAU
QTY_FACTOR_FOR_AVERAGE = 1.65
EVE_MAX_ORDER_COUNT = 20

df[CN['Close']] = pd.to_datetime(df[CN['Close']])
df[CN['Open']] = pd.to_datetime(df[CN['Open']])
df[CN['OpenValue']] = (df['Price'] * 100) * df['Size'] # *100 for USD To Cents
df[CN['BalanceOut']] = df[CN['Profit']].cumsum()
df[CN['BalanceIn']] = df[CN['BalanceOut']] - df[CN['Profit']]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828 entries, 0 to 827
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Deal            828 non-null    int64         
 1   Open time       828 non-null    datetime64[ns]
 2   Type            828 non-null    object        
 3   Size            828 non-null    float64       
 4   Item            814 non-null    object        
 5   Price           828 non-null    float64       
 6   S/L             828 non-null    int64         
 7   T/P             828 non-null    float64       
 8   Time            828 non-null    datetime64[ns]
 9   Close           828 non-null    float64       
 10  Commission      828 non-null    int64         
 11  Swap            828 non-null    float64       
 12  Profit          828 non-null    float64       
 13  Comment         828 non-null    object        
 14  DK_OPEN_VALUE   828 non-null    float64       
 15  DK_BAL

# Chapter 1. Account Summary

In [20]:
# Account Summary

df_sum = df

df_sum['Days'] = df_sum[CN['Open']].dt.date
df_sum['Balance'] = df_sum[CN['Profit']] / 100
df_sum['Deposit'] = 0
df_sum['Withdrawal'] = 0
df_sum['Profit'] = 0
df_sum.loc[df_sum['Comment'].str.contains('Deposit', regex=False), ['Deposit']] = df_sum['Balance']
df_sum.loc[df_sum['Comment'].str.contains('Withdrawal', regex=False), ['Withdrawl']] = df_sum['Balance']
df_sum.loc[df_sum['Comment'].str.contains(COMMENT_PATTERN_FOR_ORDERS_ONLY, regex=False), ['Profit']] = df_sum['Balance']

df_sum = df_sum.groupby(lambda x: True).agg({
    'Days': pd.Series.nunique,
    'Balance': 'sum',
    'Deposit': 'sum',
    'Withdrawal': 'sum',
    'Profit': 'sum',
})

df_sum.style.format({
  'Balance': lambda val: '${val:,.2f}',
  'Deposit': lambda val: f'${val:,.2f}',
  'Withdrawal': lambda val: f'${val:,.2f}',
  'Profit': lambda val: f'${val:,.2f}',
})

# pd.options.display.float_format = '${:,.2f}'.format



df_sum



KeyError: "Column(s) ['Date'] do not exist"

In [112]:
# Set each row NET_ID

# Filter only orders deals without deposits
df_o = df[df[CN['Comment']].str.contains(COMMENT_PATTERN_FOR_ORDERS_ONLY, regex=False)]

# Set DK_NET_ID equal Deal for all first net orders
df_o.loc[df[CN['Comment']].str.contains(COMMENT_PATTERN_FOR_NET_START), CN['NetID']] = df_o[CN['OrderID']]

df_o = df_o.sort_values(by=[CN['Close'], CN['Open']], ascending=True)

net_id_list = []
qty_list = []
qty_cum = 0
val_list = []
val_cum = 0
close_dt_prev = None
for (order_id, close_dt, qty, val) in zip(df_o[CN['OrderID']], df_o[CN['Close']], df_o[CN['Qty']], df_o[CN['OpenValue']]):
    if close_dt_prev != close_dt:
        net_id_list.append(order_id)
        close_dt_prev = close_dt
        qty_cum = qty
        val_cum = val
    else:
        net_id_list.append(net_id_list[-1])
        qty_cum = qty_cum + qty
        val_cum = val_cum + val

    qty_list.append(qty_cum)
    val_list.append(val_cum)

df_o[CN['NetID']] = net_id_list
df_o[CN['NetID']] = df_o[CN['NetID']].astype(int)
df_o[CN['NetQty']] = qty_list
df_o[CN['NetValue']] = val_list

#todo change loop for shift vector
df_o = df_o.sort_values(by=[CN['NetID'], CN['Open']], ascending=False)
worst_price_list = []
net_id_prev = None
open_price_prev = None
for (net_id, open_price, side) in zip(df_o[CN['NetID']], df_o[CN['OpenPrice']], df_o[CN['Side']]):
    if net_id == net_id_prev:
        worst_price_list.append(open_price_prev)
    else:
        max_price_delta = (OPEN_NEW_ORDER_PRICE_DELTA_PIPS - 1) * XAU_PIP_USD
        if side == 'buy': 
            max_price_delta = -1 * max_price_delta
        worst_price_list.append(open_price + max_price_delta)

    open_price_prev = open_price
    net_id_prev = net_id


df_o[CN['WorstPrice']] = worst_price_list
df_o[CN['Drawdown']] = abs(df_o[CN['NetQty']] * (df_o[CN['WorstPrice']] * 100) - df_o[CN['NetValue']]) # *100 for USD price to Cents
df_o[CN['DrawdownRatio']] = df_o[CN['Drawdown']] / df_o[CN['BalanceIn']]

df_o = df_o.sort_values(by=CN['Open'], ascending=True)


/var/folders/9m/tth9zlt910549vvdqgrwg7rh0000gn/T/ipykernel_2433/1903683029.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_o.loc[df[CN['Comment']].str.contains(COMMENT_PATTERN_FOR_NET_START), CN['NetID']] = df_o[CN['OrderID']]


In [113]:
x = df_o[df_o[CN['NetID']] == 72718771]
# x = df_o
x.head(20)

,Deal,Open time,Type,Size,Item,Price,S/L,T/P,Time,Close,...,Comment,DK_OPEN_VALUE,DK_BALANCE_OUT,DK_BALANCE_IN,DK_NET_ID,DK_NET_QTY,DK_NET_VALUE,DK_WORST_PRICE,DK_DRAWDOWN,DK_DRAWDOWN_RATIO
782,72718771,2023-06-15 15:29:02,sell,0.15,XAUUSD,1926.29,0,1953.22,2023-06-15 17:55:07,1953.22,...,Start SELL[tp],28894.35,1049771.57,1050175.52,72718771,0.15,28894.35,1928.60,34.65,0.000033
783,72722096,2023-06-15 15:31:00,sell,0.17,XAUUSD,1928.60,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 2[tp],32786.20,1049353.03,1049771.57,72718771,0.32,61680.55,1930.83,106.01,0.000101
784,72724914,2023-06-15 15:33:01,sell,0.26,XAUUSD,1930.83,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 3[tp],50201.58,1048770.89,1049353.03,72718771,0.58,111882.13,1932.63,210.41,0.000201
789,72732143,2023-06-15 15:44:00,sell,0.40,XAUUSD,1932.63,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 4[tp],77305.20,1048034.71,1048858.31,72718771,0.98,189187.33,1935.62,503.43,0.000480
792,72741000,2023-06-15 16:01:00,sell,0.62,XAUUSD,1935.62,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 5[tp],120008.44,1047010.94,1048102.14,72718771,1.60,309195.77,1937.79,850.63,0.000812
794,72744870,2023-06-15 16:09:01,sell,0.98,XAUUSD,1937.79,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 6[tp],189903.42,1045521.30,1047033.44,72718771,2.58,499099.19,1940.02,1425.97,0.001362
797,72753460,2023-06-15 16:26:08,sell,1.58,XAUUSD,1940.02,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 7[tp],306523.16,1043483.10,1045568.70,72718771,4.16,805622.35,1942.92,2632.37,0.002518
799,72755691,2023-06-15 16:28:01,sell,2.61,XAUUSD,1942.92,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 8[tp],507102.12,1040819.70,1043508.00,72718771,6.77,1312724.47,1945.25,4209.78,0.004034
801,72762225,2023-06-15 16:42:01,sell,4.31,XAUUSD,1945.25,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 9[tp],838402.75,1037407.43,1040842.50,72718771,11.08,2151127.22,1948.45,7755.38,0.007451
803,72768254,2023-06-15 16:47:15,sell,7.11,XAUUSD,1948.45,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 10[tp],1385347.95,1034038.61,1037430.08,72718771,18.19,3536475.17,1951.58,13448.85,0.012964


In [116]:
# TOP5 nets with biggest drawdown
df_o[CN['NetLastPrice']] = df_o['Price']

df_nets = df_o.groupby([CN['NetID']]).agg({
  CN['Open']: 'min',
  CN['Close']: 'max',
  CN['OrderID']: 'count',
  CN['Profit']: 'sum',
  CN['Qty']: 'sum',
  CN['OpenPrice']: 'first',
  CN['NetLastPrice']: 'last',
  CN['OpenValue']: 'sum',
  CN['WorstPrice']: 'last',
  CN['BalanceIn']: 'min',
}).reset_index()

df_nets[CN['NetAvgPrice']] = df_nets[CN['OpenValue']] / df_nets[CN['Qty']]
df_nets[CN['Drawdown']] = abs(df_nets[CN['Qty']] * (df_nets[CN['WorstPrice']] * 100) - df_nets['DK_OPEN_VALUE'])  # *100 for USD price to Cents
df_nets[CN['DrawdownRatio']] = df_nets[CN['Drawdown']] / df_nets[CN['BalanceIn']]
df_nets = df_nets.sort_values(by=CN['DrawdownRatio'], ascending=False)

def calc_dd(r):
    kq = QTY_FACTOR_FOR_AVERAGE
    kp = (OPEN_NEW_ORDER_PRICE_DELTA_PIPS - 1) * XAU_PIP_USD
    
    curr_price = r[CN['NetLastPrice']]
    curr_value = r[CN['OpenValue']]
    curr_qty = r[CN['Qty']]
    for i in range(r[CN['OrderID']] + 1, EVE_MAX_ORDER_COUNT + 1):
      curr_price = curr_price + kp
      curr_value = curr_value + (curr_price * 100) * (curr_qty * kq - curr_qty)
      curr_qty = curr_qty * kq

    return abs(curr_value - curr_qty * (curr_price + kp)  * 100)


df_nets['DK_DRAWDOWN_20'] = df_nets.apply(calc_dd, axis = 1)
df_nets['DK_DRAWDOWN_20_RATIO'] = df_nets['DK_DRAWDOWN_20'] / df_nets[CN['BalanceIn']]

df_nets.head()

,DK_NET_ID,Open time,Time,Deal,Profit,Size,Price,DK_NET_LAST_PRICE,DK_OPEN_VALUE,DK_WORST_PRICE,DK_BALANCE_IN,DK_NET_AVG_PRICE,DK_DRAWDOWN,DK_DRAWDOWN_RATIO,DK_DRAWDOWN_20,DK_DRAWDOWN_20_RATIO
359,72718771,2023-06-15 15:29:02,2023-06-15 17:55:07,14,19589.28,130.83,1926.29,1958.57,25573566.54,1960.36,1032138.14,195471.730796,73823.34,0.071525,1.213971e+06,1.176171
200,71462744,2023-06-07 16:54:00,2023-06-07 20:52:00,14,9044.57,58.57,1968.93,1941.51,11391547.36,1939.72,518315.22,194494.576746,30607.32,0.059052,5.007823e+05,0.966173
30,70146186,2023-05-30 08:54:02,2023-05-30 16:00:20,13,2401.49,15.69,1932.26,1959.51,3069137.51,1961.30,205769.57,195611.058636,8142.19,0.039569,2.383875e+05,1.158517
213,71560632,2023-06-07 22:58:01,2023-06-08 20:12:32,13,6412.12,42.71,1940.43,1969.08,8394100.89,1970.87,631970.51,196537.131585,23484.88,0.037161,6.502392e+05,1.028908
144,70963762,2023-06-02 22:28:02,2023-06-05 12:41:38,6,3344.17,14.88,1950.14,1939.90,2890428.95,1938.11,210795.36,194249.257392,6521.27,0.030936,7.487565e+06,35.520538


In [21]:
# df_o['DK_OPEN_VALUE'] = df_o['Price'] * df_o['Size']
# df_o['DK_CLOSE_VALUE'] = df_o['DK_WORST_PRICE'] * df_o['Size'] - df_o['DK_OPEN_VALUE']
# df_o['DK_LOSS'] = df_o['DK_CLOSE_VALUE'] - df_o['DK_OPEN_VALUE']

x = df_o[df_o[CN['NetID']] == 71535499]
x = df_o[df_o[CN['NetID']] == 72780472]
x .head(20)

,Deal,Open time,Type,Size,Item,Price,S/L,T/P,Time,Close,Commission,Swap,Profit,Comment,DK_BALANCE_OUT,DK_BALANCE_IN,DK_NET_ID,DK_WORST_PRICE,DK_OPEN_VALUE
782,72718771,2023-06-15 15:29:02,sell,0.15,XAUUSD,1926.29,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-403.95,Start SELL[tp],1049771.57,1050175.52,72780472,1928.60,288.9435
783,72722096,2023-06-15 15:31:00,sell,0.17,XAUUSD,1928.60,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-418.54,SELL 2[tp],1049353.03,1049771.57,72780472,1930.83,327.8620
784,72724914,2023-06-15 15:33:01,sell,0.26,XAUUSD,1930.83,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-582.14,SELL 3[tp],1048770.89,1049353.03,72780472,1932.63,502.0158
789,72732143,2023-06-15 15:44:00,sell,0.40,XAUUSD,1932.63,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-823.60,SELL 4[tp],1048034.71,1048858.31,72780472,1935.62,773.0520
792,72741000,2023-06-15 16:01:00,sell,0.62,XAUUSD,1935.62,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-1091.20,SELL 5[tp],1047010.94,1048102.14,72780472,1937.79,1200.0844
794,72744870,2023-06-15 16:09:01,sell,0.98,XAUUSD,1937.79,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-1512.14,SELL 6[tp],1045521.30,1047033.44,72780472,1940.02,1899.0342
797,72753460,2023-06-15 16:26:08,sell,1.58,XAUUSD,1940.02,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-2085.60,SELL 7[tp],1043483.10,1045568.70,72780472,1942.92,3065.2316
799,72755691,2023-06-15 16:28:01,sell,2.61,XAUUSD,1942.92,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-2688.30,SELL 8[tp],1040819.70,1043508.00,72780472,1945.25,5071.0212
801,72762225,2023-06-15 16:42:01,sell,4.31,XAUUSD,1945.25,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-3435.07,SELL 9[tp],1037407.43,1040842.50,72780472,1948.45,8384.0275
803,72768254,2023-06-15 16:47:15,sell,7.11,XAUUSD,1948.45,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-3391.47,SELL 10[tp],1034038.61,1037430.08,72780472,1951.58,13853.4795


In [3]:
import datetime

df_o.groupby([df_o[COLUMN_NAME_OPEN_DATETIME].dt.date]).agg({
  COLUMN_NAME_CLOSE_DATETIME: 'max',
  COLUMN_NAME_NET_ID: pd.Series.nunique,
  COLUMN_NAME_DEAL: 'count',
  'Size': 'sum',
  'DK_OPEN_VALUE': 'sum'
  ''
}).reset_index()

NameError: name 'df_o' is not defined

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(df_o[['Time']])
# plt.scatter(df.index[df.Buy_Signal], df[df.Buy_Signal].Close, marker = '^', color='g')
# plt.scatter(df.index[df.Sell_Signal], df[df.Sell_Signal].Close, marker = 'v', color='r')
# plt.fill_between(df.index, df.Upper, df.Lower, color='grey', alpha=0.3)
# plt.legend(['Close', 'SMA', 'Upper', 'Lower'])
plt.show()